In [1]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

In [2]:
## umap csv
df = pd.read_csv('umap_appended.csv')
df2 = pd.read_csv('tsne_appended.csv')

## Split into dataframes based on labels (umap)
df_umap_1 = df.loc[df['labels'] == 0]
df_umap_2 = df.loc[df['labels'] == 1]
df_umap_3 = df.loc[df['labels'] == 2]
df_umap_1_describe = df_umap_1.describe()
df_umap_2_describe = df_umap_2.describe()
df_umap_3_describe = df_umap_3.describe()

In [3]:
df_merged = df = pd.merge(df,df2[['icustay_id','labels']],on='icustay_id', how='left')
df_merged.head()
df_tsne_2 = df2.loc[df2['labels'] == 1]
print(df_tsne_2.shape)
print(df_umap_2.shape)
## find icustay_id that is in both df_umap_0 and df_tsne_2
df_merged_icustay_id = df_tsne_2.loc[~df_tsne_2['icustay_id'].isin(df_umap_2['icustay_id'])]
print(df_merged_icustay_id.shape)
print(df_merged_icustay_id['icustay_id'])


(135, 344)
(125, 344)
(11, 344)
996     209881
2207    221594
2224    221720
2663    226013
4521    244588
5095    250103
5612    255146
7726    275725
7766    276135
8468    282744
9002    288246
Name: icustay_id, dtype: int64


In [4]:
fig_compare = px.scatter(
    df, x='umap_0', y='umap_1',
    color=df2['labels'],
    hover_data=['icustay_id'],
    title = "UMAP vs TSNE label comparison"
)
fig_compare.show()


In [5]:
df_umap_1_describe.head()

,Unnamed: 0,icustay_id,creatinine0,fraction_inspired_oxygen0,heart_rate0,hematocrit0,mean_blood_pressure0,respiratory_rate0,sodium0,creatinine1,...,hematocrit47,mean_blood_pressure47,respiratory_rate47,sodium47,readmission_30,mort_hosp,mort_icu,labels,umap_0,umap_1
count,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,...,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,7172.0,7172.000000,7172.000000
mean,5090.240100,249892.573759,1.444597,0.733455,90.646712,31.545506,99.202747,18.784255,138.018372,1.444513,...,30.125971,78.991733,19.824096,138.634900,0.072225,0.163413,0.120608,0.0,-1.371427,8.152859
std,2940.817028,28855.088307,1.503958,0.255992,20.451980,6.347405,1419.571418,6.236126,5.263458,1.503722,...,4.816558,70.233154,5.884169,5.018439,0.258879,0.369768,0.325694,0.0,1.597230,3.822965
min,0.000000,200001.000000,0.000000,0.000000,32.750000,9.400000,-34.000000,0.000000,0.000000,0.000000,...,8.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,-4.476324,-1.203622
25%,2538.750000,224859.500000,0.700000,0.500000,77.000000,27.000000,70.000000,14.500000,135.000000,0.700000,...,26.800000,68.000000,16.000000,136.000000,0.000000,0.000000,0.000000,0.0,-2.785925,5.337839


In [6]:
df_umap_1.shape

(7172, 344)

In [7]:
subsetDataFrame = df_umap_1[df_umap_1['respiratory_rate2'] > 100]
subsetDataFrame.head()
subsetDataFrame['respiratory_rate2']

5818    1177788.5
Name: respiratory_rate2, dtype: float64

In [8]:
df_umap_1_describe['respiratory_rate2']

count    7.172000e+03
mean     1.830226e+02
std      1.390722e+04
min      0.000000e+00
25%      1.466667e+01
50%      1.800000e+01
75%      2.200000e+01
max      1.177788e+06
Name: respiratory_rate2, dtype: float64

In [11]:
## sampling function for each label
def sample_plot(df_label, df_describe, n, features, y_label):
    sample = df_label.sample(n)
    new_df = sample.loc[:, sample.columns.to_series().str.contains(features)]
    transpose  = new_df.transpose()
    transpose.columns = ['sample'+'_{}'.format(i) for i in range(len(transpose.columns))]
    ## mean median
    mean_df = df_describe.loc[['mean'], df_describe.columns.to_series().str.contains(features)]
    median_df = df_describe.loc[['50%'], df_describe.columns.to_series().str.contains(features)]
    stdev_df = df_describe.loc[['std'], df_describe.columns.to_series().str.contains(features)]
    mean_Transpose = mean_df.transpose()
    median_Transpose = median_df.transpose()
    stdev_Transpose = stdev_df.transpose()
    ## merge the two dataframes
    mean_median_Transpose = pd.merge(mean_Transpose, median_Transpose, left_index=True, right_index=True)
    mean_median_Transpose = pd.merge(mean_median_Transpose, stdev_Transpose, left_index=True, right_index=True)
    mean_median_Transpose.columns = ['mean', 'median', 'stdev']
    print(transpose.head())

    fig = go.Figure()
    for i in range(len(transpose.columns)):
        fig.add_trace(go.Scatter(x=transpose.index, y=transpose.iloc[:,i], name=transpose.columns[i], line = dict(width=2, dash='dash')))
    fig.add_trace(go.Scatter(x=mean_median_Transpose.index, y=mean_median_Transpose['mean'], name='mean', line=dict(color='firebrick', width=2)))
    fig.add_trace(go.Scatter(x=mean_median_Transpose.index, y=mean_median_Transpose['median'], name='median', line=dict(color='royalblue', width=2)))
    # fig.add_trace(go.Scatter(x=mean_median_Transpose.index, y=mean_median_Transpose['stdev'], name='stdev', line=dict(color='green', width=2)))
    fig.update_layout(xaxis_title='Time', yaxis_title=y_label)
    fig.show()
        

In [12]:
sample_plot(df_umap_3, df_umap_3_describe, 5, "fraction_inspired_oxygen", "fraction_inspired_oxygen")

                           sample_0  sample_1  sample_2  sample_3  sample_4
fraction_inspired_oxygen0      0.21      0.21      0.21      0.21      0.21
fraction_inspired_oxygen1      0.21      0.21      0.21      0.21      0.21
fraction_inspired_oxygen2      0.21      0.21      0.21      0.21      0.21
fraction_inspired_oxygen3      0.21      0.21      0.21      0.21      0.21
fraction_inspired_oxygen4      0.21      0.21      0.21      0.21      0.21


In [8]:
def find_mean_median(df, features):
    mean_df = df.loc[['mean'], df.columns.to_series().str.contains(features)]
    median_df = df.loc[['50%'], df.columns.to_series().str.contains(features)]
    mean_Transpose = mean_df.transpose()
    median_Transpose = median_df.transpose()
    mean_median_Transpose = pd.merge(mean_Transpose, median_Transpose, left_index=True, right_index=True)
    mean_median_Transpose.columns = ['mean', 'median']
    return mean_median_Transpose

def plot_feature_label(features, label1, label2, label3, y_label, stat='mean'):
    mean_median_1 = find_mean_median(label1, features)
    mean_median_2 = find_mean_median(label2, features)
    mean_median_3 = find_mean_median(label3, features)
    ## plot mean for different labels
    if stat == 'mean':
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=mean_median_1.index, y=mean_median_1['mean'], name = 'label 1' , line=dict(color='firebrick', width=2)))
        fig.add_trace(go.Scatter(x=mean_median_2.index, y=mean_median_2['mean'], name = 'label 2' ,line=dict(color='royalblue', width=2)))
        fig.add_trace(go.Scatter(x=mean_median_3.index, y=mean_median_3['mean'], name = 'label 3' ,line=dict(color='green', width=2)))
        fig.update_layout(xaxis_title='Time', yaxis_title=y_label, title=y_label+'_mean')
        fig.show()
    ## plot median for different labels
    elif stat == 'median':
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=mean_median_1.index, y=mean_median_1['median'], name = 'label 1' ,line=dict(color='firebrick', width=2)))
        fig.add_trace(go.Scatter(x=mean_median_2.index, y=mean_median_2['median'], name = 'label 2' ,line=dict(color='royalblue', width=2)))
        fig.add_trace(go.Scatter(x=mean_median_3.index, y=mean_median_3['median'], name = 'label 3' ,line=dict(color='green', width=2)))
        fig.update_layout(xaxis_title='Time', yaxis_title=y_label, title=y_label+'_median')
        fig.show()


In [9]:
plot_feature_label('respiratory_rate', df_umap_1_describe, df_umap_2_describe, df_umap_3_describe, 'respiratory_rate', 'mean')

In [14]:
plot_feature_label('heart_rate', 'label_1', 'label_2', 'label_3', 'heart_rate', 'median')

In [3]:
## Label analysis function
def plot_features(features, df, y_label):
    mean_df = df.loc[['mean'], df.columns.to_series().str.contains(features)]
    median_df = df.loc[['50%'], df.columns.to_series().str.contains(features)]
    mean_Transpose = mean_df.transpose()
    median_Transpose = median_df.transpose()
    ## merge the two dataframes
    mean_median_Transpose = pd.merge(mean_Transpose, median_Transpose, left_index=True, right_index=True)
    mean_median_Transpose.columns = ['mean', 'median']
    ## plot the dataframe
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=mean_median_Transpose.index, y=mean_median_Transpose['mean'], name='mean', mode='lines'))
    fig.add_trace(go.Scatter(x=mean_median_Transpose.index, y=mean_median_Transpose['median'], name='median', mode='lines'))
    fig.update_layout(xaxis_title='Time', yaxis_title=y_label)
    return fig

## randomly sample 10 points from each label and plot against the mean and median for each feature
def random_sample(features, df, y_label):
    mean_df = df.loc[['mean'], df.columns.to_series().str.contains(features)]
    median_df = df.loc[['50%'], df.columns.to_series().str.contains(features)]
    mean_Transpose = mean_df.transpose()
    median_Transpose = median_df.transpose()
    ## merge the two dataframes
    mean_median_Transpose = pd.merge(mean_Transpose, median_Transpose, left_index=True, right_index=True)
    mean_median_Transpose.columns = ['mean', 'median']
    ## randomly sample 10 points from each label and plot against the mean and median for each feature
    

In [4]:
sample_2 = df_umap_2.sample(n=5)
new_df = sample_2.loc[:, sample_2.columns.to_series().str.contains('heart_rate')]
new_df.head()

,heart_rate0,heart_rate1,heart_rate2,heart_rate3,heart_rate4,heart_rate5,heart_rate6,heart_rate7,heart_rate8,heart_rate9,...,heart_rate38,heart_rate39,heart_rate40,heart_rate41,heart_rate42,heart_rate43,heart_rate44,heart_rate45,heart_rate46,heart_rate47
7974,78.666667,78.666667,78.666667,78.666667,64.75,62.5,63.5,63.5,59.0,59.000000,...,68.0,71.000000,73.0,73.0,71.0,93.0,98.0,100.0,104.0,87.0
1988,78.000000,78.000000,78.000000,78.000000,78.00,78.0,78.0,78.0,78.0,81.142857,...,84.0,81.000000,82.0,82.0,82.0,89.0,83.0,91.0,79.0,80.0
2304,80.250000,80.250000,80.250000,80.250000,77.00,80.0,80.0,80.0,80.5,81.200000,...,123.0,109.666667,115.5,113.2,112.5,121.0,112.0,124.0,111.0,101.0
6511,60.333333,60.333333,60.333333,60.333333,72.00,68.0,70.0,71.0,78.0,82.000000,...,72.0,74.000000,78.0,83.0,88.0,97.0,82.0,86.0,92.0,94.0
7077,80.000000,80.000000,87.833333,87.500000,90.00,79.5,67.5,90.0,77.0,74.000000,...,85.0,81.000000,81.0,86.0,80.0,79.0,93.0,78.0,75.0,74.0


In [5]:
transpose  = new_df.transpose()
transpose.head()
## UPDATE COLUMNS NAMES with for loop
transpose.columns = ['sample'+'_{}'.format(i) for i in range(len(transpose.columns))]
transpose.head()


,sample_0,sample_1,sample_2,sample_3,sample_4
heart_rate0,78.666667,78.0,80.25,60.333333,80.000000
heart_rate1,78.666667,78.0,80.25,60.333333,80.000000
heart_rate2,78.666667,78.0,80.25,60.333333,87.833333
heart_rate3,78.666667,78.0,80.25,60.333333,87.500000
heart_rate4,64.750000,78.0,77.00,72.000000,90.000000


In [6]:
mean_df = df_umap_2_describe.loc[['mean'], df_umap_2_describe.columns.to_series().str.contains('heart_rate')]
median_df = df_umap_2_describe.loc[['50%'], df_umap_2_describe.columns.to_series().str.contains('heart_rate')]
mean_Transpose = mean_df.transpose()
median_Transpose = median_df.transpose()
## merge the two dataframes
mean_median_Transpose = pd.merge(mean_Transpose, median_Transpose, left_index=True, right_index=True)
mean_median_Transpose.columns = ['mean', 'median']

In [7]:
## plot the dataframe
fig = go.Figure()
for i in range(len(transpose.columns)):
    fig.add_trace(go.Scatter(x=transpose.index, y=transpose.iloc[:,i], name=transpose.columns[i], line = dict(width=2, dash='dash')))
fig.add_trace(go.Scatter(x=mean_median_Transpose.index, y=mean_median_Transpose['mean'], name='mean', line=dict(color='firebrick', width=2)))
fig.add_trace(go.Scatter(x=mean_median_Transpose.index, y=mean_median_Transpose['median'], name='median', line=dict(color='royalblue', width=2)))
fig.update_layout(xaxis_title='Time', yaxis_title='Heart Rate')
fig.show()

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_median_Transpose.index, y=mean_median_Transpose['mean'], name='mean', mode='lines'))
fig.add_trace(go.Scatter(x=mean_median_Transpose.index, y=mean_median_Transpose['median'], name='median', mode='lines'))
fig.update_layout(xaxis_title='Time', yaxis_title='Heart Rate')
fig.show()

In [ ]:
## Label analysis function
def plot_features(features, df, y_label):
    mean_df = df.loc[['mean'], df.columns.to_series().str.contains(features)]
    median_df = df.loc[['50%'], df.columns.to_series().str.contains(features)]
    mean_Transpose = mean_df.transpose()
    median_Transpose = median_df.transpose()
    ## merge the two dataframes
    mean_median_Transpose = pd.merge(mean_Transpose, median_Transpose, left_index=True, right_index=True)
    mean_median_Transpose.columns = ['mean', 'median']
    ## plot the dataframe
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=mean_median_Transpose.index, y=mean_median_Transpose['mean'], name='mean', mode='lines'))
    fig.add_trace(go.Scatter(x=mean_median_Transpose.index, y=mean_median_Transpose['median'], name='median', mode='lines'))
    fig.update_layout(xaxis_title='Time', yaxis_title=y_label)
    return fig